# 10 Data aggregation and group

## 10.1 GroupBy Mechanics

### 10.1.1 Iterating over groups

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                  'key2':['one','two','one','two','one'],
                  'data1':np.random.randn(5),
                  'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.354527,1.348273
1,a,two,-1.910394,0.015828
2,b,one,0.488570,0.710593
3,b,two,-0.371693,-1.296770
4,a,one,-0.208982,-0.042487


In [7]:
# compute the group means
means = df.groupby([df['key1'],df['key2']]).mean()
means

data1     data2
key1 key2                    
a    one   0.572772  0.652893
     two  -1.910394  0.015828
b    one   0.488570  0.710593
     two  -0.371693 -1.296770

In [10]:
# pass column names as group keys (more directly)
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one   0.572772  0.652893
     two  -1.910394  0.015828
b    one   0.488570  0.710593
     two  -0.371693 -1.296770

In [12]:
# get a Series counting group size
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.2 Iterating over group

In [16]:
# iterate the group name and data
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.354527  1.348273
1    a  two -1.910394  0.015828
4    a  one -0.208982 -0.042487
b
  key1 key2     data1     data2
2    b  one  0.488570  0.710593
3    b  two -0.371693 -1.296770


In [18]:
# iterate the multiple keys and data
for(k1,k2),group in df.groupby(['key1','key2']):
    print(k1,k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one  1.354527  1.348273
4    a  one -0.208982 -0.042487
a two
  key1 key2     data1     data2
1    a  two -1.910394  0.015828
b one
  key1 key2    data1     data2
2    b  one  0.48857  0.710593
b two
  key1 key2     data1    data2
3    b  two -0.371693 -1.29677


In [27]:
# compute a dict of the data pieces 
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.488570,0.710593
3,b,two,-0.371693,-1.296770


In [28]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [30]:
# group the columns of the dataframe by dtype
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.354527  1.348273
1 -1.910394  0.015828
2  0.488570  0.710593
3 -0.371693 -1.296770
4 -0.208982 -0.042487
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.3 Selecting a column or subset of colums

In [69]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.652893
     two   0.015828
b    one   0.710593
     two  -1.296770

In [71]:
#return aggregate data without row indexes
df.groupby(['key1','key2'],as_index=False)[['data2']].mean()

,key1,key2,data2
0,a,one,0.652893
1,a,two,0.015828
2,b,one,0.710593
3,b,two,-1.296770


### 10.1.5 Grouping by index levels

In [38]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                    [1,3,5,1,3]],
                                   names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.385264 -0.991469  0.510918  0.842834 -1.482450
1     -0.438912 -0.592444  0.307853 -0.633024  0.122322
2      0.336346  1.236706  0.515586 -0.288960 -0.892934
3     -0.844597  0.111008 -0.376802  3.166345  2.089104

In [39]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data aggregation

In [49]:
df

,key1,key2,data1,data2
0,a,one,1.354527,1.348273
1,a,two,-1.910394,0.015828
2,b,one,0.488570,0.710593
3,b,two,-0.371693,-1.296770
4,a,one,-0.208982,-0.042487


In [45]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.041825
b    0.402543
Name: data1, dtype: float64

In [46]:
def peak_to_peak(arr):
    return arr.max()-arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,3.264921,1.390760
b,0.860262,2.007363


### 10.2.1 Column-wise and multiple function application

In [48]:
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [63]:
# aggregate to calculate the group mean, standard deviation and differences
tips.groupby(['day','smoker'])[['tip_pct']].agg(['mean','std',peak_to_peak])

tip_pct                       
                 mean       std peak_to_peak
day  smoker                                 
Fri  No      0.151650  0.028123     0.067349
     Yes     0.174783  0.051293     0.159925
Sat  No      0.158048  0.039767     0.235193
     Yes     0.147906  0.061375     0.290095
Sun  No      0.160113  0.042347     0.193226
     Yes     0.187250  0.154134     0.644685
Thur No      0.160298  0.038774     0.193350
     Yes     0.163863  0.039389     0.151240

In [65]:
# set name for the columns
tips.groupby(['day','smoker'])[['tip_pct']].agg([('foo','mean'),('bar',np.std)])

tip_pct          
                  foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

## 10.3 Apply: general split-apply-combine

In [73]:
def top(df, n =5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
